In [275]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb
%run ./4-ImplementationBP.ipynb

./img/3/graph.html
./img/3/graph.html


In [276]:
def delta_funct(S_ki, t_i):
    if (len(S_ki) == 0):
        return 0
    if (t_i == np.min(S_ki)):
        return 1
    return 0

In [277]:
def R_i(t_i, r_i, lambda_i):
    if (t_i > r_i):
        return 0
    result = lambda_i*np.exp(-lambda_i*(r_i - t_i))
    return result

In [278]:
def S_ij(s_ij, t_i, r_i, lambda_i, time_range, contact_range_ij):
    if (t_i <= s_ij < r_i):
            temp = np.where((contact_range_ij < s_ij) & (contact_range_ij >= t_i))
            return (lambda_i)*(1-lambda_i)**(len(temp))
    elif (s_ij ==9):
        index_larger_than_t_i = np.where(contact_range_ij >= t_i)
        return (1-lambda_i)**(len(index_larger_than_t_i))
    else:
        return 0

In [279]:
#debug function
def Find_Zero(delt, R, S):
    if (delt == 0):
        print("delt = 0")
    if (R == 0):
        print("R = 0")
    if (S == 0):
        print("S = 0")

def phi(t_i,r_i, s, type):
    delt = delta_funct([s[1-type]],t_i)
    R = R_i(t_i, r_i, 0.3)
    S = S_ij(s[type], t_i, r_i, 0.3, "nothing", np.array([0,6,9]))
    #Find_Zero(delt, R, S)
    return delt*R*S


In [280]:
#phi(t_i,r_i,(s_ij,s_ji), type)

In [281]:
phi(1, 7, (6,1), 0)

0.010413829957959952

In [282]:
#inference using BP

In [283]:
variables_i = ["t_i","r_i","s_ij:s_ji"]
variables_j = ["t_j", "r_j","s_ij:s_ji"]

In [284]:
shape = (10,10,10,10)
matrix_phi_i = np.zeros(shape=(10,10,10))
matrix_phi_j = np.zeros(shape=(10,10,10))
for index in np.ndindex(shape):
    matrix_phi_i[index[0], index[1], index[2]] = phi(index[0], index[1], (index[2], index[3]),0)
    matrix_phi_j[index[0], index[1], index[3]] = phi(index[0], index[1], (index[2], index[3]),1)

In [285]:
phi_i = factor(variables_i, matrix_phi_i)
phi_j = factor(variables_j, matrix_phi_j)

In [286]:
graph = factor_graph()
graph.add_factor_node("phi_i", phi_i)
graph.add_factor_node("phi_j", phi_j)

In [287]:
bp = belief_propagation(graph)
bp_result = bp.belief("t_i").get_distribution()

In [288]:
bp_result

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [289]:
plot_factor_graph(graph)

./img/3/graph.html


In [290]:
#exact calculation of marginals

In [291]:
t_i_array = []
for t_i in range(10):
    sum = 0
    for r_i in range(10):
        for t_j in range(10):
            for r_j in range(10):
                for s_ij in range(10):
                    for s_ji in range(10):
                        sum += phi_i.get_distribution()[t_i, r_i, s_ij]*phi_j.get_distribution()[t_j, r_j, s_ji]
    t_i_array.append(sum)

In [292]:
sum_Z = np.sum(t_i_array)

In [293]:
t_i_array

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04409999999999999]

In [294]:
marginal_t_i = t_i_array/sum_Z
marginal_t_i

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [295]:
bp_result

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [296]:
factor_i = factor(["t_i", "t_j"], np.array([[0.1, 0.2], [0.3, 0.4]]))
factor_j = factor(["t_j", "t_k"], np.array([[0.5, 0.6], [0.7, 0.8]]))

graph = factor_graph()
graph.add_factor_node("factor_i", factor_i)
graph.add_factor_node("factor_j", factor_j)

bp = belief_propagation(graph)
bp_result = bp.belief("t_i").get_distribution()

In [297]:
bp_result

array([0.30597015, 0.69402985])

In [298]:
#exact calculation of marginals
exact = []
for t_i in range(2):
    sum = 0
    for t_j in range(2):
        for t_k in range(2):
            sum += factor_i.get_distribution()[t_i, t_j]*factor_j.get_distribution()[t_j, t_k]
    exact.append(sum)

sum_Z = np.sum(exact)
exact

[0.41000000000000003, 0.9299999999999999]

In [299]:
exact/sum_Z

array([0.30597015, 0.69402985])